In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pandas as pd
import numpy as np
import keras
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def NN(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized

    # define model (with dropout)
    initializer = keras.initializers.HeNormal(seed=1)

    regressor_peak = keras.models.Sequential()
    regressor_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(50,  activation='relu'))
    regressor_peak.add(keras.layers.Dropout(space['x1']))
    regressor_peak.add(Dense(1, kernel_initializer=initializer))

    # Compile model

    opt = keras.optimizers.Adam(learning_rate=space['x3'])
    regressor_peak.compile(loss='mean_squared_error', optimizer= opt)

    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe PL Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)


    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y = df['FWHM_eV']

    from sklearn.model_selection import GroupShuffleSplit 

    splitter = GroupShuffleSplit(test_size=.1, n_splits=2, random_state = 1)
    split = splitter.split(df, groups=df['Expt. No'])
    train_inds, test_inds = next(split)

    train = df.iloc[train_inds]
    test = df.iloc[test_inds]

    X_train = train[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y_train = train['FWHM_eV']

    X_test = test[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    y_test = test['FWHM_eV']

    X_train = K.constant(X_train)
    y_train = K.constant(y_train)
    X_test = K.constant(X_test)
    y_train = K.constant(y_train)
            
    matrix = np.zeros([10,13])
    i = 0
    j = 0
        
    #reset weights of model
    regressor_peak.load_weights('initialize_weights3.h5')
    
    #prediction for Peak
    regressor_peak.fit(X_train, y_train, epochs=int(space['x2']), validation_data=(X_test, y_test),verbose = 0, use_multiprocessing=False, workers = 100)
    y_pred = regressor_peak.predict(X_test)

    #errors in eV
    matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
    matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
    matrix[10*i + j,2] = rae(y_test, y_pred)
    matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
    matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = 10*matrix.mean(0)[:8]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0])
    print("R2 (norm, eV):", matrix_2[0,1])
    print("RAE (norm, eV):", matrix_2[0,2])
    print("RMSE (norm, eV):", matrix_2[0,3])
    print("MAE (nm):", matrix_2[0,4])
    print("R2 (nm):", matrix_2[0,5])
    print("RAE (nm):", matrix_2[0,6])
    print("RMSE (nm):", matrix_2[0,7])

    return {'loss': matrix_2[0,3], 'status': STATUS_OK }

In [7]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = {'x1': hp.uniform('x1', 0.1,0.5),
         'x2': hp.quniform('x2', 10,600,20),
         'x3': hp.uniform('x3', 0.0001,0.01)
        }

best = fmin(NN,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

5/5 [==============================] - 0s 2ms/step                                                                    

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.025887874604936068                                                                                                   
R2 (norm, eV):                                                                                                         
0.3684824158954433                                                                                                     
RAE (norm, eV):                                                                                                        
0.6142573559566695                                                                                                     
RMSE (norm, eV):                                

MAE (norm, eV):                                                                                                        
0.025306465441379086                                                                                                   
R2 (norm, eV):                                                                                                         
0.4496422390804149                                                                                                     
RAE (norm, eV):                                                                                                        
0.6004619068908201                                                                                                     
RMSE (norm, eV):                                                                                                       
0.041898957663983286                                                                                                   
MAE (nm):                               

0.5293116867707071                                                                                                     
RAE (norm, eV):                                                                                                        
0.5833654533148667                                                                                                     
RMSE (norm, eV):                                                                                                       
0.03874782941325064                                                                                                    
MAE (nm):                                                                                                              
5.6713668751198885                                                                                                     
R2 (nm):                                                                                                               
0.5495578090065252                      

RMSE (norm, eV):                                                                                                       
0.043120767818783996                                                                                                   
MAE (nm):                                                                                                              
6.2217832872135865                                                                                                     
R2 (nm):                                                                                                               
0.4359861433396227                                                                                                     
RAE (nm):                                                                                                              
0.6382810649574546                                                                                                     
RMSE (nm):                              

6.044229209306021                                                                                                      
R2 (nm):                                                                                                               
0.39655779012985437                                                                                                    
RAE (nm):                                                                                                              
0.6200661254935096                                                                                                     
RMSE (nm):                                                                                                             
9.945711581218596                                                                                                      
5/5 [==============================] - 0s 0s/step                                                                     

Absorption Peak                         

...

6.263119971658499                                                                                                      
R2 (nm):                                                                                                               
0.42486334745755117                                                                                                    
RAE (nm):                                                                                                              
0.6425217177978598                                                                                                     
RMSE (nm):                                                                                                             
9.709649231107878                                                                                                      
5/5 [==============================] - 0s 0s/step                                                                     

Absorption Peak                         

RAE (nm):                                                                                                              
0.5626436270572515                                                                                                     
RMSE (nm):                                                                                                             
8.904926273321626                                                                                                      
5/5 [==============================] - 0s 3ms/step                                                                    

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.02520312788086983                                                                                                    
R2 (norm, eV):                                  

8.899283758645145                                                                                                      
5/5 [==============================] - 0s 2ms/step                                                                    

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.024418808952950254                                                                                                   
R2 (norm, eV):                                                                                                         
0.3619608337295893                                                                                                     
RAE (norm, eV):                                                                                                        
0.5793999411674495                              

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.024519975812811602                                                                                                   
R2 (norm, eV):                                                                                                         
0.4067935283674943                                                                                                     
RAE (norm, eV):                                                                                                        
0.5818003888209246                                                                                                     
RMSE (norm, eV):                                                                                                       
0.04349943455329957                             

R2 (norm, eV):                                                                                                         
0.35077420117230296                                                                                                    
RAE (norm, eV):                                                                                                        
0.7092583204068339                                                                                                     
RMSE (norm, eV):                                                                                                       
0.04550703668500356                                                                                                    
MAE (nm):                                                                                                              
6.939926156937231                                                                                                      
R2 (nm):                                